In [4]:
import pandas as pd


file_path = 'Ingredients.csv'


try:
    df = pd.read_csv(file_path, header=None, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, header=None, encoding='ISO-8859-1')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, header=None, encoding='windows-1252')

In [9]:
row = df.iloc[9]
print(row)


0    c("fresh mushrooms", "butter", "boneless skinl...
Name: 9, dtype: object


In [10]:
import re

def extract_items(input_string):
    
    items = re.findall(r'"([^"]+)"', input_string)
    return items

unique_items = set()


for index, row in df.iterrows():
    items = extract_items(row[0])
    unique_items.update(items)

In [11]:
unique_items

{'chicken legs',
 'tuna',
 'Okra Freezer Gumbo Starter',
 'Harry Potter Spell Books',
 'chocolate chips',
 'beef broth',
 'peppercorn',
 'chicken breast fillet',
 'Keto Cauliflower Crust',
 'tamari soy sauce',
 'plain fat-free yogurt',
 'sweet potatoes in syrup',
 'fat-free sugar-free instant pistachio pudding mix',
 'fresh pumpkin puree',
 'fresh chervil',
 'hamburger meat',
 'cajun-style stewed tomatoes',
 "The Last Red Enchilada Sauce You'll Need",
 'chicken breasts',
 'swiss chard leaves',
 'ground hazelnuts',
 'bottled lemon juice',
 'catfish fillets',
 'bonito flakes',
 'meyer lemon zest',
 'parmesan croutons',
 'low-fat granola cereal with raisins',
 'red sweet bell peppers',
 'marsala',
 'Chinese mustard',
 'beef sirloin steaks',
 'low-sodium chicken bouillon cubes',
 'raw egg yolks',
 'green chili powder',
 'marshmallow peeps',
 '2% fat cottage cheese',
 'Jello Instant Vanilla Pudding Mix',
 'herb-seasoned stuffing cubes',
 'carbonated water',
 'lime wedges',
 'TABASCO® brand 

In [12]:
import mysql.connector


conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='admin123',
    database='food_recommendation_db'
)
cursor = conn.cursor()


cursor.execute('''
CREATE TABLE IF NOT EXISTS ingredient (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    Description VARCHAR(255)
)
''')


for item in unique_items:
    try:
        cursor.execute('INSERT INTO ingredient (name) VALUES (%s)', (item,))
    except mysql.connector.errors.IntegrityError:
        continue


conn.commit()


cursor.close()
conn.close()
